In [ ]:
!pip install pandas scikit-learn torch transformers datasets matplotlib

# 1. **Install and Import Required Libraries**

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    precision_recall_curve
)

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset as HFDataset

# 2. **Merge CSV Files for Train, Validation, and Test**

In [ ]:
# Change paths accordingly
DATA_DIR = "/content/"

# --- 2.1 Read each dataset's CSVs ---
# MT-CSD
mtcsd_train = pd.read_csv(os.path.join(DATA_DIR, "MT-CSD_train.csv"))
mtcsd_dev   = pd.read_csv(os.path.join(DATA_DIR, "MT-CSD_dev.csv"))
mtcsd_test  = pd.read_csv(os.path.join(DATA_DIR, "MT-CSD_test.csv"))

# VAST
vast_train = pd.read_csv(os.path.join(DATA_DIR, "VAST_train.csv"))
vast_dev   = pd.read_csv(os.path.join(DATA_DIR, "VAST_dev.csv"))
vast_test  = pd.read_csv(os.path.join(DATA_DIR, "VAST_test.csv"))

# Russia-Ukraine_War
ruw_train = pd.read_csv(os.path.join(DATA_DIR, "Russia-Ukraine_War_train.csv"))
ruw_dev   = pd.read_csv(os.path.join(DATA_DIR, "Russia-Ukraine_War_dev.csv"))
ruw_test  = pd.read_csv(os.path.join(DATA_DIR, "Russia-Ukraine_War_test.csv"))

# Various_Tweets_(2023)
vt_train = pd.read_csv(os.path.join(DATA_DIR, "Various_Tweets_(2023)_train.csv"))
vt_val   = pd.read_csv(os.path.join(DATA_DIR, "Various_Tweets_(2023)_test.csv"))
vt_test  = pd.read_csv(os.path.join(DATA_DIR, "Various_Tweets_(2023)_test.csv"))

# --- 2.2 Concatenate train, dev (validation), and test sets ---
train_df = pd.concat([mtcsd_train, vast_train, ruw_train, vt_train], ignore_index=True)
val_df   = pd.concat([mtcsd_dev, vast_dev, ruw_dev, vt_val], ignore_index=True)
test_df  = pd.concat([mtcsd_test, vast_test, ruw_test, vt_test], ignore_index=True)

# Sanity check
print("Train size:", len(train_df))
print("Val size:  ", len(val_df))
print("Test size: ", len(test_df))

# Ensure columns are the same in each df
print("Columns:", train_df.columns.tolist())

# 3. **Prepare the Data for Training**

In [ ]:
def map_stance_to_label(stance):
    """
    Original stance values: -1 (Against), 0 (None), 1 (Favor)
    New label values: 0 -> 'Against', 1 -> 'None', 2 -> 'Favor'
    """
    if stance == -1:
        return 0
    elif stance == 0:
        return 1
    elif stance == 1:
        return 2
    else:
        raise ValueError(f"Unexpected stance: {stance}")

def combine_inputs(row):
    return f"Target: {row['Target']} [SEP] Context: {row['Context']} [SEP] Text: {row['Text']}"

train_df["combined_text"] = train_df.apply(combine_inputs, axis=1)
val_df["combined_text"]   = val_df.apply(combine_inputs, axis=1)
test_df["combined_text"]  = test_df.apply(combine_inputs, axis=1)

train_df["label"] = train_df["Stance"].apply(map_stance_to_label)
val_df["label"]   = val_df["Stance"].apply(map_stance_to_label)
test_df["label"]  = test_df["Stance"].apply(map_stance_to_label)

In [ ]:
#Create Huggingface Datasets
hf_train = HFDataset.from_pandas(train_df[['combined_text', 'label']])
hf_val   = HFDataset.from_pandas(val_df[['combined_text', 'label']])
hf_test  = HFDataset.from_pandas(test_df[['combined_text', 'label']])

# 4. **Tokenize the Data**

In [ ]:
from datasets import Features, Value

model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(
        example["combined_text"],
        padding="max_length",
        truncation=True,
        max_length=128  # Adjust as needed
    )

hf_train = hf_train.map(tokenize_function, batched=True)
hf_val   = hf_val.map(tokenize_function, batched=True)
hf_test  = hf_test.map(tokenize_function, batched=True)

hf_train = hf_train.remove_columns(["combined_text"])
hf_val   = hf_val.remove_columns(["combined_text"])
hf_test  = hf_test.remove_columns(["combined_text"])

# Cast "labels" to int64
from datasets import Value
hf_train = hf_train.cast_column("label", Value("int64"))
hf_val   = hf_val.cast_column("label", Value("int64"))
hf_test  = hf_test.cast_column("label", Value("int64"))


# 5. **Create the Model**

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 6. **Define Training Arguments and Trainer**

In [ ]:
training_args = TrainingArguments(
    output_dir="output_dir",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    num_train_epochs=3,            # Adjust
    per_device_train_batch_size=8, # Adjust
    per_device_eval_batch_size=8,  # Adjust
    warmup_steps=100,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="wandb"
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    precision = precision_score(labels, preds, average="macro")
    recall = recall_score(labels, preds, average="macro")
    f1 = f1_score(labels, preds, average="macro")
    accuracy = accuracy_score(labels, preds)

    return {
        "accuracy": accuracy,
        "macro_precision": precision,
        "macro_recall": recall,
        "macro_f1": f1
    }




trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
#Train the model
trainer.train()

# 7. **Evaluate on Validation**

In [ ]:
val_preds_output = trainer.predict(hf_val)
val_logits = val_preds_output.predictions
val_labels = val_preds_output.label_ids
val_preds = np.argmax(val_logits, axis=-1)

print("Validation classification report:")
print(classification_report(val_labels, val_preds, digits=3))

# 8. **Evaluate on Test**



In [ ]:
test_preds_output = trainer.predict(hf_test)
test_logits = test_preds_output.predictions
test_labels = test_preds_output.label_ids
test_preds = np.argmax(test_logits, axis=-1)

print("Test classification report:")
print(classification_report(test_labels, test_preds, digits=3))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Confusion matrix
cm = confusion_matrix(test_labels, test_preds)

# Update the labels
labels = [-1, 0, 1]

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix (on Test)')
plt.show()

# 9. **Precision-Recall Curves**

In [ ]:
def plot_precision_recall_curves(logits, labels, num_classes=3):
    """Plot a separate Precision-Recall curve for each of the remapped classes (-1, 0, 1) in the correct order."""
    probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()

    # Mapping of original class indices to new ones
    class_mapping = {1: -1, 0: 0, 2: 1}  # Original 0->0, 1->-1, 2->1
    remapped_classes = sorted(class_mapping.values())  # Ensure the order is [-1, 0, 1]

    # Reverse the mapping to align indices with remapped_classes
    reverse_mapping = {v: k for k, v in class_mapping.items()}

    fig, axes = plt.subplots(1, num_classes, figsize=(5 * num_classes, 5), sharey=True)

    for idx, remapped_class in enumerate(remapped_classes):
        original_class = reverse_mapping[remapped_class]  # Get original class index
        # Binarize: current class vs rest
        y_true = (labels == original_class).astype(int)
        y_scores = probs[:, original_class]

        precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
        axes[idx].plot(recall, precision, label=f"Class {remapped_class}")
        axes[idx].set_title(f"Precision-Recall Curve (Class {remapped_class})")
        axes[idx].set_xlabel("Recall")
        axes[idx].set_ylabel("Precision")
        axes[idx].legend(loc="lower left")
        axes[idx].grid(True)

    plt.tight_layout()
    plt.show()

# Plot for the test set
plot_precision_recall_curves(test_logits, test_labels, num_classes=3)